In [ ]:
%matplotlib inline
import numpy as np
import nidaqmx as ni
import matplotlib.pyplot as plt
import time

## Set up DAQ

The following cell initialize the DAQ device for acepting the external io. Do this only once when the DAQ is on.

In [ ]:
if 0:
    read_trigger.close()
    write_voltage.close()
    write_done.close()
    read_test.close()

read_trigger=ni.task.Task(new_task_name='task1')
read_trigger.ci_channels.add_ci_count_edges_chan('Dev1/ctr0',edge=ni.constants.Edge['RISING'])
write_voltage=ni.task.Task(new_task_name='task2')
write_voltage.ao_channels.add_ao_voltage_chan('Dev1/ao0:1', max_val=10, min_val=-10)
write_done=ni.task.Task(new_task_name='task3')
write_done.do_channels.add_do_chan('Dev1/port0/line7')
#read_test=ni.task.Task(new_task_name='task4')
#read_test.di_channels.add_di_chan('Dev1/port0/line2')

In [ ]:
read_trigger.start()
write_done.start()
write_voltage.start()

## Create voltage list

In [ ]:
# xy_cent=np.array([0.295, 0.228]) #volts 20180123
xy_cent=np.array([0.525,-0.046]) #volts #volts 20180124

volt_per_deg=0.5
max_NA=0.35
r_led=6 #radius, in unit of "# of leds"

[NA_x,NA_y]=np.meshgrid(np.linspace(-max_NA,max_NA,2*r_led+1),np.linspace(-max_NA,max_NA,2*r_led+1))
NA_x[np.arange(1,NA_x.shape[0],2),:]=NA_x[np.arange(1,NA_x.shape[0],2),::-1] #Flip every other column
#Limit down to a circle
valid_led=(NA_x**2+NA_y**2)<=(max_NA**2)

NA_x=NA_x[valid_led]
NA_y=NA_y[valid_led]

angleD_x=np.arcsin(NA_x)*180/np.pi
angleD_y=np.arcsin(NA_y)*180/np.pi

v_x=angleD_x*volt_per_deg/2 + xy_cent[0]
v_y=angleD_y*volt_per_deg/2 + xy_cent[1]

num_galvo=len(v_x)


In [ ]:
#Save it
num_galvo

In [ ]:
xy_cent=np.array([0.525,-0.046]) #volts#volts 
write_voltage.write(xy_cent)

In [ ]:
v_x[:]=xy_cent[0]
v_y[:]=xy_cent[1]

## Galvo control

In [ ]:
#Set back to 0,0
write_voltage.write(xy_cent)

write_done.write(True)

counter_cur=read_trigger.read()
counter_val=counter_cur #Init counter

for vv in range(num_galvo):
    print('Waiting for FPGA signal.')
    while int(counter_cur) <=int(counter_val):
        counter_cur=read_trigger.read()
        print( '\b\b'+ str(counter_cur),end='')
        read_trigger.wait_until_done()
    print('Writing voltage to galvo.')
    counter_val=counter_cur #Replace value when incremented
    success=write_voltage.write(np.array([v_x[vv],v_y[vv]])) #Write voltage
    if success == 0:
        print('Failed to write, stopping.')
        break
    print('Sending done signal.')
    #time.sleep(5)
    success=write_done.write(False)
    if success == 0:
        print('Failed to write, stopping.')
        break
    time.sleep(0.005)
    success=write_done.write(True)
    if success == 0:
        print('Failed to write, stopping.')
        break
    

In [ ]:
read_trigger.stop()
write_voltage.stop()
write_done.stop()

read_trigger.close()
write_voltage.close()
write_done.close()
#read_test.close()